# Final Assignment
## CM3015 Machine Learning and Neural Networks

### Credit Card Fraud Detection with a Feedforward MLP

- Student: cy150
- Workflow: Chollet's ML workflow (problem → data → evaluation → prep → baseline → model → tuning → final eval)

---

## Step 1 — Define the problem

Chollet’s workflow keeps the project aligned with real‑world goals. It moves in clear stages from problem definition to data understanding, evaluation design, preparation, baseline, model building, tuning, and final reporting.

### Overview: Credit Card Fraud

Credit card fraud is the unauthorized use of a credit (or debit) card to make purchases, withdraw funds, or create transactions that the legitimate cardholder did not approve.

### Problem Statement


- Credit card fraud causes direct and significant financial losses: issuers, merchants, and consumers may absorb losses from unauthorized purchases and chargebacks. Fraud also creates investigative overhead (reviews, disputes) and temporary loss of funds and reputation. As a result, stricter verification and KYC are implemented by merchants and banks.

-  These organizations can also lose customers if there are excessive false declines or chargebacks. Additionally, failure to protect customers according to regulatory standards can incur penalties to banks themselves proving that credit card fraud detection is a critical technology for a functioning, safe banking and finance environment
---

## Step 2 — Identify and understand the data

### Dataset Overview
This dataset consists of credit card transactions by European cardholders. It covers two days of transactions with 492 frauds out of 284,807 transactions.
The dataset is highly imbalanced, with frauds accounting for about 0.172% of all transactions.


### Nature of the Dataset

It contains only numerical input variables resulting from a PCA transformation. Due to confidentiality, the original features and more background information are not available.

### Dataset Features
Features V1, V2, … V28 are principal components from PCA. The only features not transformed with PCA are `Time` and `Amount`.

- `Time` contains the seconds elapsed between each transaction and the first transaction in the dataset.

*Example*

| Time |      V1 |      V2 |     V3 |     V4 |      V5 |
| ---: | ------: | ------: | -----: | -----: | ------: |
|    0 | -1.3598 | -0.0728 | 2.5363 | 1.3782 | -0.3383 |
|    1 |  1.1919 |  0.2662 | 0.1665 | 0.4482 |  0.0600 |
|    1 | -1.3584 | -1.3402 | 1.7732 | 0.3798 | -0.5032 |


### Dataset Licensing

The dataset is licensed under the Database Contents License (DbCL).

According to the license (Open Data Commons), the Licensor grants a worldwide, royalty-free, non-exclusive, perpetual, irrevocable copyright license to do any act that is restricted by copyright over anything within the Contents, whether in the original medium or any other. These rights explicitly include commercial use and do not exclude any field of endeavor.

### Permission

The DbCL license explicitly allows use of this dataset for this final assignment.

### Dataset Author

- Machine Learning Group - ULB 

### Dataset Source

After browsing Kaggle, I selected a dataset that is complex and challenging while providing rich features for the model to learn from.

Link to dataset: `https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data`

### Justification for this dataset

This dataset supports a full end-to-end deep learning workflow: clear labels, numeric features, and a real-world class imbalance that demands careful evaluation.

#### Rationale behind why this dataset was chosen

- It matches the **problem framing**: a real-world binary classification task where the positive class (fraud) is rare and costly to miss.
- It matches the **data modality**: fixed-length, fully numeric, tabular features that are well-suited to a feedforward MLP as a strong first baseline.
- It forces a realistic **evaluation setup**: extreme imbalance means accuracy is not meaningful, so the workflow naturally prioritizes PR AUC, precision/recall, and explicit threshold selection.
- It encourages good **experimental discipline**: preprocessing must be fit on the training split only (to avoid leakage) and the final test set can be kept untouched for a single, final report.

#### Limitations of this dataset and mitigation strategy

- **Limited interpretability**: V1–V28 are anonymized PCA components, so feature-level explanations are not meaningful.
  - *Mitigation*: focus on predictive performance, stability across runs, and careful threshold selection rather than per-feature interpretation.
- **Potential time effects**: `Time` reflects ordering within the 2-day window.
  - *Mitigation*: use a time-aware split (train on earlier, validate/test on later) to better reflect deployment.

---

## Step 1 Cont'd — Define success metrics

### Objective

The primary objective is to detect fraudulent transactions while minimizing false positives. Because fraud is rare, the evaluation focuses on minority-class performance and selecting an operating point that reflects the cost of errors.

## Step 3 — Choose an evaluation protocol

### Holdout Protocol

1. The data will be split into:

    - Training set
    - Validation set
    - Test set

   **Why**: we need separate data for learning parameters (train), choosing settings (validation), and an unbiased final report (test).

   **How**: use a time-aware split when possible (earlier transactions → train, later → validation/test) so evaluation better matches deployment.

2. Preprocessing decisions are fitted using the training set exclusively.

   **Why**: using any information from validation/test (even feature scaling statistics) leaks signal and inflates performance.

   **How**: fit transforms on `X_train` only (e.g., standardization mean/std, any imputation rules), then apply the fitted transforms unchanged to `X_val` and `X_test`.

3. The validation set is only used for model and threshold selection.

   **Why**: the validation set simulates unseen data during development; using it only for selection reduces the risk of overfitting the final report.

   **How**: compare candidate models using PR AUC/recall/precision on validation; choose hyperparameters and pick an operating threshold (e.g., maximize recall subject to minimum precision) using validation predictions.

4. Final performance is evaluated on the untouched test set.

   **Why**: the test set should be a single, unbiased estimate of how the chosen pipeline will perform in the real world.

   **How**: once preprocessing + model + threshold are finalized, run inference once on `X_test` and report the locked metrics (PR AUC and confusion matrix at the chosen threshold).


### Evaluation metrics

This section describes how performance will be measured based on the pre-defined success criteria.

### Primary Metrics

| Metric               | What it measures                                                       | Why it matters for fraud under heavy class imbalance                                                    |
| -------------------- | ---------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------- |
| Precision–Recall AUC | Area under the precision–recall curve across decision thresholds       | Strong overall summary metric when fraud is rare, more informative than accuracy                        |
| Recall               | True positive rate, how many actual fraud cases are correctly detected | Directly captures missed fraud risk since low recall means more fraud slips through                     |
| F1 Score             | Harmonic mean of precision and recall                                  | Useful single number when you want a balanced tradeoff between catching fraud and limiting false alarms |


### Secondary Metric

| Secondary metric                              | Description                                                                                                       | Purpose                                                                                        |
| --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------- |
| Confusion matrix at a predetermined threshold | Uses TP, FP, TN, FN to make tradeoffs explicit                                                                    | Shows performance at the chosen operating point and clarifies the cost of each type of mistake |
| Error rates                                   | False negative rate and false positive rate to quantify misses and false alarms                                   | Measures miss risk versus false alarm burden in a comparable way                               |
| Calibration check                             | Compare predicted probabilities with observed outcomes using a simple binning table to verify probability quality | Checks whether predicted risk scores align with real observed fraud rates                      |


#### Justification for evaluation metrics

Fraud is a rare event, so a single metric can be misleading. A model may look strong on one metric while failing in practice. The primary metric provides a consistent rule for model comparison, while secondary metrics provide the context needed to interpret false-positive and false-negative tradeoffs.

### Implications of the evaluation metrics

1. Precision reflects workload and customer friction.
2. Recall reflects loss prevention, often translating into direct financial losses.
3. PR AUC reflects ranking quality under rare fraud across thresholds.
4. The confusion matrix is reported at an operating threshold aligned with transaction behavior.


## Step 4 — Prepare the data

### Data preparation plan

To prepare the data and avoid leakage:

1. Use a time-aware split: earlier transactions for training, later transactions for validation/test.
2. Fit all preprocessing steps (scaling, imputation if needed) on the training set only.
3. Apply the same fitted transforms to validation and test sets.
4. Preserve the class imbalance during splitting to reflect real deployment.
5. Track feature distributions and label rate over time to identify drift.
6. Use a small threshold sweep on the validation set for later operating-point selection.
7. Calibrate probabilities with simple binning to sanity-check outputs.
8. Re-train periodically as base rates drift in production.

Data checks and class imbalance

- Check for missing values, big outliers, and changes between the train/val/test splits.
- Scale features using numbers from the training set only.
- Handle imbalance with class weights or resampling.
- Keep notes of every preprocessing step so results are repeatable.

---

In [1]:
import sys

# Install into THIS kernel environment (Python 3.9.6 in your case)
%pip -q install -U "kagglehub[pandas-datasets]"

import kagglehub
from kagglehub import KaggleDatasetAdapter

print("Kernel Python:", sys.version)
print("Kernel executable:", sys.executable)

# For this dataset, the file inside the dataset is:
file_path = "creditcard.csv"

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "mlg-ulb/creditcardfraud",
    file_path,
)

print(df.head())


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Kernel Python: 3.9.6 (default, Dec  2 2025, 07:27:58) 
[Clang 17.0.0 (clang-1700.6.3.2)]
Kernel executable: /Library/Developer/CommandLineTools/usr/bin/python3


/Users/yanguangchen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yanguangchen/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/f9/k1h51sjx6tg89fbj4gy3c1k00000gn/T/ipykernel_67714/4066093214.py:16: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Resuming download from 0 bytes (69155672 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/mlg-ulb/creditcardfraud?dataset_version_number=3&file_name=creditcard.csv (0/69155672) bytes left.


100%|██████████| 66.0M/66.0M [00:04<00:00, 16.5MB/s]

Extracting zip of creditcard.csv...


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

### Step 4 implementation (code)

The code below loads the dataset from `data/creditcard.csv`.
If the file is missing, it will try to download it using `kagglehub` and then save it to `data/creditcard.csv`.

- Expected columns include `Time`, `Amount`, `V1`…`V28`, and `Class` (target label).


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

DATA_PATH = Path("data/creditcard.csv")

# If the CSV isn't present locally, try fetching it via kagglehub.
if not DATA_PATH.exists():
    try:
        import kagglehub
        from kagglehub import KaggleDatasetAdapter

        df = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            "mlg-ulb/creditcardfraud",
            "creditcard.csv",
        )
        DATA_PATH.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(DATA_PATH, index=False)
        print("Downloaded via kagglehub and saved to:", DATA_PATH)
    except Exception as e:
        raise FileNotFoundError(
            f"Could not find {DATA_PATH} and kagglehub download failed.\n"
            "Either place the file at data/creditcard.csv, or configure Kaggle access for kagglehub, then rerun.\n"
            f"Original error: {type(e).__name__}: {e}"
        ) from e
else:
    df = pd.read_csv(DATA_PATH)
    print("Loaded:", DATA_PATH)

print("Shape:", df.shape)
print(df.head(3))

if "Class" not in df.columns:
    raise ValueError("Expected a 'Class' column (0=legit, 1=fraud).")

fraud_rate = df["Class"].mean()
print(f"Fraud rate: {fraud_rate:.6f} ({df['Class'].sum()} / {len(df)})")


/var/folders/f9/k1h51sjx6tg89fbj4gy3c1k00000gn/T/ipykernel_67714/2749449166.py:14: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Downloaded via kagglehub and saved to: data/creditcard.csv
Shape: (284807, 31)
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  

[3 rows x 31 columns]
Fraud rate: 0.001727 (492 / 284807)


In [3]:
try:
    from sklearn.preprocessing import StandardScaler
except ImportError as e:
    raise ImportError(
        "scikit-learn is required for StandardScaler. Install it (e.g., `pip install scikit-learn`) and rerun."
    ) from e

# Time-aware ordering (closer to deployment): train on earlier, test on later
if "Time" in df.columns:
    df = df.sort_values("Time").reset_index(drop=True)

feature_names = [c for c in df.columns if c != "Class"]
X = df[feature_names].to_numpy(dtype=np.float32)
y = df["Class"].to_numpy(dtype=np.int64)

n = len(df)
train_end = int(0.70 * n)
val_end = int(0.85 * n)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

print("Split sizes:")
print("- train:", X_train.shape, "fraud_rate=", float(y_train.mean()))
print("- val:  ", X_val.shape, "fraud_rate=", float(y_val.mean()))
print("- test: ", X_test.shape, "fraud_rate=", float(y_test.mean()))

# Fit preprocessing on TRAIN ONLY to avoid leakage
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_val_scaled = scaler.transform(X_val).astype(np.float32)
X_test_scaled = scaler.transform(X_test).astype(np.float32)

# Optional: class weights for imbalanced training
neg = int((y_train == 0).sum())
pos = int((y_train == 1).sum())
class_weight = {0: 1.0, 1: (neg / max(pos, 1))}
print("class_weight:", class_weight)


Split sizes:
- train: (199364, 30) fraud_rate= 0.0019261250777472363
- val:   (42721, 30) fraud_rate= 0.001310830738980829
- test:  (42722, 30) fraud_rate= 0.0012171714807359207
class_weight: {0: 1.0, 1: 518.1770833333334}


### Split Success

The split ran successfully because we created **three non-overlapping subsets** (train/val/test) with clear row counts and the same number of input features (30 columns). The sizes are sensible (about 70% / 15% / 15%), and the **fraud rate stays low and in the expected range** across all splits.

The computed `class_weight` is large for class 1, which confirms fraud is rare in the training set and will be up-weighted during training. Finally, scaling is fit on `X_train` only and then applied to validation/test, which helps prevent data leakage.

## Step 5 — Establish a baseline and pick a starting model

A trivial baseline is to always predict non-fraud; this sets the minimum acceptable recall and PR AUC. After establishing the baseline, I start with a small-to-medium feedforward MLP: fully connected Dense layers with Dropout and a single sigmoid output for binary classification.

#### Model Size
- Small to medium feedforward multilayer perceptron

## Step 6 — Develop the model

### Overview of feedforward neural networks

A feedforward network passes inputs through a stack of Dense layers without cycles. For tabular data, this is an effective first-choice architecture.

### Overview of multilayer perceptrons

A multilayer perceptron (MLP) stacks Dense layers with nonlinear activations (e.g., ReLU) to learn complex decision boundaries from tabular numeric features.

### Justification: Why a feedforward MLP

- Works well for fixed-length tabular numeric inputs.
- Efficient to train and tune compared to more complex architectures.
- Provides a strong baseline before exploring more specialized models.

## Step 7 — Model improvement and threshold tuning

- Instead of using a standard 0.5 cutoff, choose a decision threshold on the validation set (maximize recall subject to a minimum precision).
- Use PR AUC as the primary model selection metric, which is more appropriate than accuracy for rare-event detection.
- Run each configuration multiple times and report mean and standard deviation for stability.
- Use a time-aware split to mirror real deployment conditions.
- Apply binning-based calibration checks to verify probability quality.


## Step 8 — Final evaluation and deployment considerations

### Current real-life applications

Fraud detection systems are used by card issuers and payment networks to rank transactions by risk, trigger manual review, or apply step-up verification. In production, models are monitored for drift, recalibrated, and re-trained as fraud patterns evolve.


## Glossary of terms


| Term                  | Definition                                                                                                             |
| --------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| Binary classification | A prediction task with two classes, here fraud versus non fraud                                                        |
| Class imbalance       | A dataset property where one class is much rarer than the other                                                        |
| Positive class        | The class of interest, here fraud transactions labeled 1                                                               |
| Negative class        | The other class, here legitimate transactions labeled 0                                                                |
| Feature               | An input variable used for prediction, such as V1 or Amount                                                            |
| Label                 | The target variable the model learns to predict, here Class                                                            |
| PCA                   | Principal Component Analysis, a transformation that creates new variables as linear combinations of original variables |
| Principal component   | One PCA derived feature, here V1 to V28                                                                                |
| Train set             | Data used to fit model parameters                                                                                      |
| Validation set        | Data used to select hyperparameters and decision threshold                                                             |
| Test set              | Held out data used once for final performance reporting                                                                |
| Data leakage          | When information from validation or test data influences training or preprocessing decisions                           |
| Standardization       | Scaling features to have zero mean and unit variance using training statistics                                         |
| Normalization         | Rescaling features to a fixed range, often 0 to 1, depending on the method                                             |
| Model                 | A function that maps input features to a predicted output                                                              |
| Neural network        | A model composed of layers of learned transformations, here Dense and Dropout layers                                   |
| Dense layer           | A fully connected layer that applies a linear transformation followed by an activation function                        |
| Dropout               | A regularization method that randomly disables a fraction of units during training to reduce overfitting               |
| Activation function   | A non linear function applied within a layer, such as ReLU or sigmoid                                                  |
| Sigmoid               | An activation that maps a real number to a value between 0 and 1, used for binary outputs                              |
| Logits                | The raw model output before applying sigmoid                                                                           |
| Probability score     | The model output after sigmoid, interpreted as probability like score                                                  |
| Decision threshold    | The cutoff used to convert probability scores into class predictions                                                   |
| Confusion matrix      | A table counting true positives false positives true negatives and false negatives                                     |
| True positive TP      | Fraud correctly predicted as fraud                                                                                     |
| False positive FP     | Legitimate predicted as fraud                                                                                          |
| True negative TN      | Legitimate correctly predicted as legitimate                                                                           |
| False negative FN     | Fraud predicted as legitimate                                                                                          |
| Precision             | TP divided by TP plus FP, the fraction of predicted fraud that is truly fraud                                          |
| Recall                | TP divided by TP plus FN, the fraction of actual fraud that is detected                                                |
| F1 score              | Harmonic mean of precision and recall                                                                                  |
| ROC curve             | Curve of true positive rate versus false positive rate over thresholds                                                 |
| AUC                   | Area under a curve, a threshold independent performance summary                                                        |
| PR curve              | Precision versus recall over thresholds                                                                                |
| PR AUC                | Area under the precision recall curve, often preferred for rare event detection                                        |
| Overfitting           | When a model performs well on training but poorly on new data                                                          |
| Regularization        | Methods that reduce overfitting, such as dropout or weight penalties                                                   |
| Hyperparameter        | A setting chosen outside training, such as number of layers, dropout rate, learning rate                               |
| Learning rate         | Step size used by the optimizer when updating model weights                                                            |
| Optimizer             | Algorithm that updates model weights to minimize the loss, such as Adam                                                |
| Loss function         | The quantity the model minimizes during training, such as binary cross entropy                                         |
| Early stopping        | Stopping training when validation performance stops improving                                                          |
| Calibration           | How well predicted probabilities match observed event rates                                                            |
| Concept drift         | When the data generating process changes over time, causing performance degradation                                    |
| Baseline model        | A simple reference model used for comparison, such as always predicting non fraud                                      |


---
### Bibliography & Citations


Carcillo, F., Le Borgne, Y. A., Caelen, O. and Bontempi, G. (2018) ‘Streaming active learning strategies for real life credit card fraud detection: assessment and visualization’, International Journal of Data Science and Analytics, 5(4), pp. 285 300.

Carcillo, F., Dal Pozzolo, A., Le Borgne, Y. A., Caelen, O., Mazzer, Y. and Bontempi, G. (2018) ‘Scarff: a scalable framework for streaming credit card fraud detection with Spark’, Information Fusion, 41, pp. 182 194.

Carcillo, F., Le Borgne, Y. A., Caelen, O., Oblé, F. and Bontempi, G. (2019) ‘Combining unsupervised and supervised learning in credit card fraud detection’, Information Sciences.

Dal Pozzolo, A. (n.d.) Adaptive machine learning for credit card fraud detection. PhD thesis. Université libre de Bruxelles.

Dal Pozzolo, A., Caelen, O., Johnson, R. A. and Bontempi, G. (2015) ‘Calibrating probability with undersampling for unbalanced classification’, in Proceedings of the IEEE Symposium on Computational Intelligence and Data Mining. IEEE.

Dal Pozzolo, A., Caelen, O., Le Borgne, Y. A., Waterschoot, S. and Bontempi, G. (2014) ‘Learned lessons in credit card fraud detection from a practitioner perspective’, Expert Systems with Applications, 41(10), pp. 4915 4928.

Dal Pozzolo, A., Boracchi, G., Caelen, O., Alippi, C. and Bontempi, G. (2018) ‘Credit card fraud detection: a realistic modeling and a novel learning strategy’, IEEE Transactions on Neural Networks and Learning Systems, 29(8), pp. 3784 3797.

Lebichot, B., Le Borgne, Y. A., He, L., Oblé, F. and Bontempi, G. (2019) ‘Deep learning domain adaptation techniques for credit cards fraud detection’, in INNSBDDL 2019 Recent Advances in Big Data and Deep Learning, pp. 78 88.

Lebichot, B., Paldino, G., Siblini, W., He, L., Oblé, F. and Bontempi, G. (n.d.) ‘Incremental learning strategies for credit cards fraud detection’, International Journal of Data Science and Analytics.

Le Borgne, Y. A. and Bontempi, G. (n.d.) Reproducible machine learning for credit card fraud detection: practical handbook.

If you want, paste your target year for the PhD thesis and the handbook, plus any missing page ranges, and I will update the entries so everything is fully complete and consistent.
